In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm

In [21]:
def calc_miss_chance(df_path: Path) -> float:
    df_orig = pd.read_csv(df_path, index_col=None)
    df_orig = df_orig.drop(['1','2','3','5','6','7'], axis=1)
    df_orig['4'] = df_orig['4'].str.replace('S0010', 'S010')
    df_orig['8'] = df_orig['8'].str.replace('S0010', 'S010')

    df = pd.DataFrame()
#     df['score'] = df_orig['0'].astype(np.float)
    df['phone_actual'] = df_orig['4'].str[6:7].astype(np.int)
    df['id_actual'] = df_orig['4'].str[9:22]
#     df['class_actual'] = df_orig['4'].str[9:13].astype(np.int)
#     df['distinct_actual'] = df_orig['4'].str[15:17].astype(np.int)
#     df['sample_actual'] = df_orig['4'].str[19:22].astype(np.int)
    df['angle_actual'] = df_orig['4'].str[28:31].astype(np.int)
    df['index_actual'] = df_orig['4'].str[38:].astype(np.int)

#     df['phone_predicted'] = df_orig['8'].str[6:7].astype(np.int)
    df['id_predicted'] = df_orig['8'].str[9:22]
#     df['class_predicted'] = df_orig['8'].str[9:13].astype(np.int)
#     df['distinct_predicted'] = df_orig['8'].str[15:17].astype(np.int)
#     df['sample_predicted'] = df_orig['8'].str[19:22].astype(np.int)
    
    df['id_match'] = df['id_actual'] == df['id_predicted']
    df_acc = df[['id_actual', 'phone_actual', 'angle_actual', 'index_actual', 'id_predicted', 'id_match']]
    df_vc = df_acc.groupby(['id_actual', 'phone_actual', 'angle_actual', 'id_predicted'])['id_match'].value_counts()
    df_vc = df_vc.rename_axis(['id_actual', 'phone_actual', 'angle_actual', 'id_predicted', 'id_match']).to_frame('counts').reset_index()
    total_samples = df_vc.set_index(['id_actual', 'phone_actual', 'angle_actual']).index.unique().__len__()
    nomiss_samples = df_vc[df_vc.id_match == True].index.__len__()
    miss_chance = 100 - nomiss_samples / (total_samples) * 100
    return miss_chance



In [ ]:
for df_path in Path('../../pipeline_results_ext/').glob('*.csv'):
    #if str(df_path).find('MSER') != -1:
    mc = calc_miss_chance(df_path)
    print(f"{df_path.stem} chance is {mc:.3f}%")

In [38]:
df_orig = pd.read_csv('../../pipeline_results_ext/MI1+resnet18+256+15.csv', index_col=None)
df_orig['1'] = df_orig['1'].str.replace('S0010', 'S010')
df_orig['2'] = df_orig['2'].str.replace('S0010', 'S010')

df = pd.DataFrame()
df['phone_actual'] = df_orig['1'].str[6:7].astype(np.int)
df['id_actual'] = df_orig['1'].str[9:22]
df['angle_actual'] = df_orig['1'].str[28:31].astype(np.int)
df['index_actual'] = df_orig['1'].str[38:].astype(np.int)

df['id_predicted'] = df_orig['2'].str[9:22]

df['id_match'] = df['id_actual'] == df['id_predicted']
df_acc = df[['id_actual', 'phone_actual', 'angle_actual', 'index_actual', 'id_predicted', 'id_match']]
df_vc = df_acc.groupby(['id_actual', 'phone_actual', 'angle_actual', 'id_predicted'])['id_match'].value_counts()
df_vc = df_vc.rename_axis(['id_actual', 'phone_actual', 'angle_actual', 'id_predicted', 'id_match']).to_frame('counts').reset_index()
total_samples = df_vc.set_index(['id_actual', 'phone_actual', 'angle_actual']).index.unique().__len__()
nomiss_samples = df_vc[df_vc.id_match == True].index.__len__()
miss_chance = 100 - nomiss_samples / (total_samples) * 100

In [40]:
dd = df_vc.pivot_table(index=['id_actual', 'phone_actual', 'angle_actual'], columns='id_match', values='counts', aggfunc='sum', fill_value=0).reset_index()

In [ ]:
print(dd[dd[True] == 0]['id_actual'].value_counts().to_frame('as').to_string())

In [43]:
df_acc.groupby(['id_actual', 'phone_actual', 'angle_actual', 'id_predicted'])['id_match'].value_counts()

id_actual      phone_actual  angle_actual  id_predicted   id_match
0001_D01_S001  1             40            0001_D01_S001  True        1
                                           0192_D01_S001  False       1
                             80            0001_D01_S001  True        1
                                           0192_D01_S001  False       1
                             200           0001_D01_S001  True        1
                                                                     ..
0898_D01_S001  3             180           0898_D01_S001  True        3
                             200           0898_D01_S001  True        2
                             260           0491_D01_S001  False       1
                             280           0151_D01_S001  False       1
                                           0741_D01_S001  False       1
Name: id_match, Length: 96978, dtype: int64

In [56]:
df_acc[df_acc.id_match == False].groupby(['id_actual', 'phone_actual', 'angle_actual', 'id_predicted']).sum()

index_actual  id_match
id_actual     phone_actual angle_actual id_predicted                         
0001_D01_S001 1            40           0192_D01_S001             4         0
                           80           0192_D01_S001             3         0
                           280          0192_D01_S001             4         0
                           300          0192_D01_S001             3         0
              2            60           0192_D01_S001             8         0
...                                                             ...       ...
0898_D01_S001 3            120          0739_D01_S001             1         0
                           160          0721_D01_S001             3         0
                           260          0491_D01_S001             7         0
                           280          0151_D01_S001             2         0
                                        0741_D01_S001             2         0

[71071 rows x 2 columns]

In [51]:
print(dd[dd[True] == 0]['id_actual'].value_counts().to_frame('as').to_string())

               as
0884_D01_S001  28
0602_D01_S001  24
0634_D01_S001  22
0049_D01_S001  22
0491_D02_S002  21
0037_D01_S005  21
0647_D01_S001  20
0352_D01_S005  20
0052_D01_S001  19
0491_D02_S003  19
0352_D01_S001  19
0118_D01_S001  19
0601_D01_S001  19
0238_D01_S002  19
0186_D01_S002  18
0896_D01_S001  18
0411_D01_S003  18
0352_D01_S002  18
0030_D01_S001  18
0238_D01_S003  18
0603_D01_S001  18
0642_D01_S001  18
0181_D01_S001  17
0119_D01_S002  17
0196_D01_S002  17
0594_D01_S001  17
0037_D01_S003  17
0037_D01_S004  17
0299_D01_S008  17
0075_D01_S002  17
0538_D01_S002  17
0669_D01_S001  17
0003_D02_S001  17
0316_D01_S001  17
0288_D01_S002  16
0496_D01_S001  16
0051_D01_S001  16
0538_D02_S001  16
0538_D01_S003  16
0194_D01_S002  16
0037_D01_S001  16
0801_D01_S001  16
0131_D01_S001  15
0292_D01_S001  15
0495_D01_S001  15
0121_D01_S001  15
0831_D01_S001  15
0003_D02_S002  15
0829_D01_S002  15
0740_D01_S001  15
0043_D01_S002  15
0119_D01_S001  15
0309_D01_S001  15
0559_D01_S001  14
0037_D01_S